In [63]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def tableau_stat_controverses(type_score):
    # Créer dossier de sortie
    os.makedirs("Tableaux", exist_ok=True)

    # Charger les fichiers
    df1 = pd.read_csv('Controversies Results Feb 24 2025.csv')
    df2 = pd.read_excel('Controversy data documentation.xlsx')

    # Filtrer df2 pour les colonnes liées au type_score
    df2_filtré = df2[df2['Pillar Tags'].apply(lambda x: type_score in str(x))]

    if df2_filtré.empty:
        print(f"Aucune ligne trouvée pour le type de score {type_score}")
        return

    colonnes_cibles = df2_filtré['Factor Column Header'].tolist()
    colonnes_cibles = [col for col in colonnes_cibles if col in df1.columns]

    if not colonnes_cibles:
        print(f"Aucune des colonnes cibles trouvées dans le fichier de données pour {type_score}")
        return
    colonnes_a_garder = ['Client_ID'] + colonnes_cibles
    df_filtré = df1[colonnes_a_garder].fillna(0)

    # Calcul des statistiques
    stats = {
        'Nom de la controverse': [],
        'Moyenne sur toutes les entreprises': [],
        'Maximum de controverses par entreprise': [],
        'Minimum de controverses par entreprise': [],
        'Nombre d entreprises concernées': []
    }

    for col in colonnes_cibles:
        if col not in df_filtré.columns:
            print(f"⚠️ Colonne absente dans df_filtré : {col}")
            continue

        try:
            valeurs = pd.to_numeric(df_filtré[col], errors='coerce').fillna(0)
        except Exception as e:
            print(f"Erreur lors du traitement de la colonne {col} : {e}")
            continue

        stats['Nom de la controverse'].append(col)
        stats['Moyenne sur toutes les entreprises'].append(valeurs.mean())
        stats['Maximum de controverses par entreprise'].append(valeurs.max())
        stats['Minimum de controverses par entreprise'].append(valeurs.min())
        stats['Nombre d entreprises concernées'].append((valeurs != 0).sum())

    df_stats = pd.DataFrame(stats)

    # Création du tableau en image
    fig, ax = plt.subplots(figsize=(20, max(2, len(df_stats) * 0.5)))
    ax.axis('off')

    table = ax.table(
    cellText=df_stats.values,
    colLabels=df_stats.columns,
    cellLoc='center',
    loc='center',
    colColours=["#d0d0d0"] + ["#f0f0f0"] * (len(df_stats.columns) - 1)
    )

    # Ajuster la taille de la police
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Ajuster la largeur des colonnes (élargir la première colonne)
    cell_dict = table.get_celld()
    for i in range(len(df_stats) + 1):  # +1 pour l'en-tête
        if (i, 0) in cell_dict:
            cell_dict[(i, 0)].set_width(0.35)  # élargir première colonne

    # Améliorer la lisibilité avec des couleurs alternées pour les lignes
    for i in range(1, len(df_stats) + 1):
        for j in range(len(df_stats.columns)):
            if (i, j) in cell_dict:
                if i % 2 == 0:
                    cell_dict[(i, j)].set_facecolor("#f9f9f9")
                else:
                    cell_dict[(i, j)].set_facecolor("#ffffff")

    # Couleur d'en-tête
    for j in range(len(df_stats.columns)):
        cell_dict[(0, j)].set_facecolor("#c0c0c0")
        cell_dict[(0, j)].set_text_props(weight='bold')


    # Enregistrement
    filename = f"Tableaux/Statistiques_controverses_{type_score}.png"
    plt.savefig(filename, bbox_inches='tight')
    plt.close()

    print(f"Tableau sauvegardé sous forme d'image : {filename}")
    return df_stats


In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def tableau_stat_controverses(type_score):
    os.makedirs("Tableaux", exist_ok=True)

    df1 = pd.read_csv('Controversies Results Feb 24 2025.csv')
    df2 = pd.read_excel('Controversy data documentation.xlsx')

    df2_filtré = df2[df2['Pillar Tags'].apply(lambda x: type_score in str(x))]

    if df2_filtré.empty:
        print(f"Aucune ligne trouvée pour le type de score {type_score}")
        return

    colonnes_cibles = df2_filtré['Factor Column Header'].tolist()
    colonnes_cibles = [col for col in colonnes_cibles if col in df1.columns]

    if not colonnes_cibles:
        print(f"Aucune des colonnes cibles trouvées dans le fichier de données pour {type_score}")
        return

    colonnes_a_garder = ['Client_ID'] + colonnes_cibles
    df_filtré = df1[colonnes_a_garder].fillna(0)

    stats = {
        'Type de controverse': [],
        'Moy.': [],
        'Max': [],
        'Min': [],
        'Nb ent. concernées': []
    }

    for col in colonnes_cibles:
        if col not in df_filtré.columns:
            continue

        try:
            valeurs = pd.to_numeric(df_filtré[col], errors='coerce').fillna(0)
        except Exception:
            continue

        stats['Type de controverse'].append(col)
        stats['Moy.'].append(round(valeurs.mean(), 4))
        stats['Max'].append(round(valeurs.max(), 4))
        stats['Min'].append(round(valeurs.min(), 4))
        stats['Nb ent. concernées'].append(int((valeurs != 0).sum()))

    df_stats = pd.DataFrame(stats)

    # Supprimer lignes vides
    colonnes_numeriques = ['Moy.', 'Max', 'Min', 'Nb ent. concernées']
    df_stats = df_stats[~(df_stats[colonnes_numeriques] == 0).all(axis=1)]

    if df_stats.empty:
        print(f"Aucune donnée pertinente pour {type_score}")
        return

    # Génération image via matplotlib
    fig, ax = plt.subplots(figsize=(10, max(2, len(df_stats) * 0.5)))  # largeur réduite ici
    ax.axis('off')

    table = ax.table(
    cellText=df_stats.values,
    colLabels=df_stats.columns,
    cellLoc='center',
    loc='center'
    )

    table.auto_set_font_size(False)
    table.set_fontsize(9)         # Taille police plus compacte
    table.scale(1, 1.2)           # Pas de zoom horizontal

    # Mise en forme (alternance, en-têtes)
    cell_dict = table.get_celld()
    n_rows = len(df_stats)
    n_cols = len(df_stats.columns)
    

    # Ajuster largeur de la première colonne uniquement (colonne 0)
    for i in range(n_rows + 1):
        if (i, 0) in cell_dict:
            cell_dict[(i, 0)].set_width(0.5)

    for i in range(1, n_rows + 1):
        for j in range(n_cols):
            cell_dict[(i, j)].set_facecolor("#f9f9f9" if i % 2 == 0 else "#ffffff")

    for j in range(n_cols):
        cell_dict[(0, j)].set_facecolor("#c0c0c0")
        cell_dict[(0, j)].set_text_props(weight='bold')

    filename = f"Tableaux/Statistiques_controverses_{type_score}.png"
    plt.savefig(filename, bbox_inches='tight', dpi=150)  # dpi pour meilleur rendu
    plt.close()

    print(f"✅ Image générée : {filename}")
    return df_stats

In [44]:
tableau_stat_controverses('E')

✅ Image générée : Tableaux/Statistiques_controverses_E.png


,Type de controverse,Moy.,Max,Min,Nb ent. concernées
0,BIODIV_LAND_USE_MODERATE_CONTROVERSIES,0.0108,3.0,0.0,1
1,BIODIV_LAND_USE_SEVERE_CONTROVERSIES,0.0144,4.0,0.0,1
4,CONTR_CLIMATE_CHNG_N_TOTAL,0.0108,2.0,0.0,2
8,CONTR_IMPACT_ECOSYSTEMS_N_TOTAL,0.0072,1.0,0.0,2
11,CONTR_TOXIC_EMISS_WASTE_N_SEVERE,0.0072,2.0,0.0,1
12,CONTR_TOXIC_EMISS_WASTE_N_TOTAL,0.0252,4.0,0.0,4
27,TOXIC_EMISS_MODERATE_CONTROVERSIES,0.0108,2.0,0.0,2
33,CONTR_ENV_IMPACT_N_SEVERE,0.0108,2.0,0.0,2
34,CONTR_ENV_IMPACT_N_TOTAL,0.0252,3.0,0.0,4


In [45]:
tableau_stat_controverses('S')

✅ Image générée : Tableaux/Statistiques_controverses_S.png


,Type de controverse,Moy.,Max,Min,Nb ent. concernées
0,CONTR_ENV_IMPACT_N_SEVERE,0.0108,2.0,0.0,2
1,CONTR_ENV_IMPACT_N_TOTAL,0.0252,3.0,0.0,4
14,COMM_REL_MODERATE_CONTROVERSIES,0.0144,3.0,0.0,2
15,COMM_REL_SEVERE_CONTROVERSIES,0.0180,3.0,0.0,3
18,CONTR_CHEM_SAFE_N_TOTAL,0.0108,1.0,0.0,3
19,CONTR_COLLECTIVE_BARG_N_SEVERE,0.0144,1.0,0.0,4
20,CONTR_COLLECTIVE_BARG_N_TOTAL,0.0719,2.0,0.0,17
21,CONTR_CONTROVERSIAL_WORKPLACE_N_SEVERE,0.0036,1.0,0.0,1
22,CONTR_CONTROVERSIAL_WORKPLACE_N_TOTAL,0.0612,2.0,0.0,15
23,CONTR_CUSTOMER_FRAUD_N_SEVERE,0.0036,1.0,0.0,1


In [46]:
tableau_stat_controverses('G')

✅ Image générée : Tableaux/Statistiques_controverses_G.png


,Type de controverse,Moy.,Max,Min,Nb ent. concernées
0,TAX_TRANSP_CONTROVERSIES_TOTAL,0.0072,1.0,0.0,2
1,BUS_ETHICS_ANTICOMP_MODERATE_CONTROVERSIES,0.0540,2.0,0.0,13
4,BUS_ETHICS_CORRUPTION_MODERATE_CONTROVERSIES,0.0468,2.0,0.0,11
5,BUS_ETHICS_CORRUPTION_SEVERE_CONTROVERSIES,0.0036,1.0,0.0,1
7,BUS_ETHICS_ETHICSFRAUD_MODERATE_CONTROVERSIES,0.0432,2.0,0.0,11
9,BUS_ETHICS_ETHICSFRAUD_VERY_SEVERE_CONTROVERSIES,0.0036,1.0,0.0,1
11,CONTR_ANTI_COMP_N_TOTAL,0.0576,2.0,0.0,14
12,CONTR_BRIB_CORRUPT_N_SEVERE,0.0036,1.0,0.0,1
13,CONTR_BRIB_CORRUPT_N_TOTAL,0.0504,2.0,0.0,11
15,CONTR_ETHICS_N_TOTAL,0.0360,2.0,0.0,9
